# Sytax Tree Builder
### This is an notebook to research all properties of syntax trees, including:
- Finding association rules

- Filling in the blank on missing word classes

- Building trees

In [1]:
import re
import nltk
import math
import requests
import ast
from bs4 import BeautifulSoup
from PyDictionary import PyDictionary

# Pre-X-Bar Theory Implementation
## This was made for the simpler units of LING486

In [2]:
x = 'The big man from n loving bagels with n'
#'The cat killed the dirty dog in the house because he threw nasty bread by the fire'
#"in the smelly and wholesome and brilliant and bubbly couch"
#"the brilliant and bubbly and whimsical ball"
p = nltk.parse.api.ParserI
_d=PyDictionary()

#Array forces ordering
syntaxRulesReverse = [
    ['(adjconj)+adj', 'adj'], 
    ['(advP)*adj', 'ajP'], 
    ['(advP)?adv', 'avP'], 
    ['PnP', 'pP'],
    ['(D)?(ajP)*N(pP)*(CP)?', 'nP'], 
    ['V(nP)+(pP)*', 'vP'], 
    ['(Nconj)+N', 'N'],
    ['(nPconj)+nP', 'nP'],
    ['nPvP', 'tP'],
    ['(tPconj)+tP', 'tP']]

In [6]:
_x = x.split()
_xd = ""
for token in _x:
    _xd += getClass(token)
_xd

/home/mkeays/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /home/mkeays/anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


Error: The Following Error occured: list index out of range
Error: The Word given is not a valid English Word
Error: The Word given is not a valid English Word
Error: The Word given is not a valid English Word
Error: The Following Error occured: list index out of range
Error: The Word given is not a valid English Word
Error: The Word given is not a valid English Word
Error: The Word given is not a valid English Word
Error: The Word given is not a valid English Word
Error: The Following Error occured: list index out of range
Error: The Word given is not a valid English Word
Error: The Word given is not a valid English Word


'XadjNXNVNXN'

In [7]:
tree = [_xd]
diagram = []
while needsParsing(tree[-1]):
    level = ""
    dlvl = []
    for rule in syntaxRulesReverse:
        m = re.search(rule[0], tree[-1])
        if m:
            level = tree[-1][0:m.start()] + rule[1] + tree[-1][m.end():]
            tree += [level]
            dlvl += [rule[1], tree[-1][m.start():m.end()], m.start(), m.end()]
            diagram += [dlvl]
            break
    if level == "":
        break
    print('.', end='')
[print(_, end='\n|\n') for _ in tree];None

.......XadjNXNVNXN
|
XajPNXNVNXN
|
XnPXNVNXN
|
XnPXnPVNXN
|
XnPXnPVnPXN
|
XnPXnPVnPXnP
|
XnPXnPvPXnP
|
XnPXtPXnP
|


In [8]:
_search = re.search(tree[-2], syntaxRulesReverse[-1][0])
tree[-2][0:_search.start()] + syntaxRulesReverse[-1][1] + tree[-2][_search.end():]

AttributeError: 'NoneType' object has no attribute 'start'

In [5]:
def needsParsing(x):
    if x != 'nP' and x != 'tP' and x != 'vP' and x != 'CP' and x!= 'pP':
        return True
    return False
def getClass(x):
    wC = 'X'
    #Meaning -- regular
    word = _d.meaning(x)
    if word is not None:
        wC = list(word.keys())[0]
    
    #Meaning -- google
    word = _d.googlemeaning(x)
    if word is not None:
        wC =  word.split(':')[1].split()[0]

    if wC.lower() == 'noun' or wC.lower() == 'pronoun':
        return 'N'
    elif wC.lower() == 'verb':
        return 'V'
    elif wC.lower() == 'adjective':
        return 'adj'
    elif wC.lower() == 'adverb':
        return 'adv'
    elif wC.lower() == 'determiner':
        return 'D'
    elif wC.lower() == 'preposition':
        return 'P'
    elif wC.lower() == 'conjunction':
        return 'conj'
        
    return wC

# X-Bar and Array Tree Data Structures

In [13]:
### Getting parts of speach using wordsmyth
word = "chopsticks"
url = "https://www.wordsmyth.net/?level=3&ent={}".format(word)
s = requests.Session()
res = s.get(url, verify = False)
soup = BeautifulSoup(res.text, 'html5lib')
soup.findAll("table", {"class": "maintable"})[0].find_all('td', {"class": "data"})[0].a.text

/home/mkeays/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


'noun'

In [15]:
def saveDict(d, filename):
    sD = str(d)
    f = open(filename, 'w')
    f.write(sD)
    f.close()
    
def loadDict(filename):
    f = open(filename, 'r')
    sD = f.read()
    f.close()
    d = ast.literal_eval(sD)
    return d

In [16]:
#temporary static dict
'''
localBank = {'is': 'v',
             'need' : 'v',
             'needs' : 'v',
             'do': 'v',
             'does': 'v',
             'I' : 'pn',
             'a' : 'd',
             'the' : 'd',
            #homework specific - no working suplementary dict
            'well': 'adj',
            'plays': 'v'}
'''

localBank = loadDict('local_bank.json')

In [40]:
saveDict(localBank, 'local_bank.dat')

In [17]:
def getLikelyClass(word, convert = False):
    if word in localBank:
        return localBank[word]
    url = "https://www.wordsmyth.net/?level=3&ent={}".format(word)
    s = requests.Session()
    res = s.get(url, verify = False)
    soup = BeautifulSoup(res.text, 'html5lib')
    txt = soup.findAll("table", {"class": "maintable"})[0].find_all('td', {"class": "data"})[0].a.text
    switch = {
        'determiner'          :        'd',
        'noun'                :        'n',
        'pronoun'             :        'pn',
        'preposition'         :        'p',
        'transitive verb'     :        'v', #vt
        'intransitive verb'  :         'v', #vi
        'ditransitive verb'  :         'v', #vd
        'adjective'           :        'adj',
        'adverb'              :        'adv',
        'conjunction'         :        'conj',
    }
    if convert and txt in switch:
        localBank[word] = switch[txt]
        return switch[txt]
    return txt

def defineLikelyClass(word, wordClass, force=False):
    if word not in localBank or force:
        localBank[word] = wordClass
        return True
    return False

In [18]:
defineLikelyClass('\'s', 'd', True)
getLikelyClass('\'s', True)

'd'

In [19]:
#For machine gunning words into the local word bank!
x = 'The dog is in the house'.split()
y = 'd n v p d n'.split()
for i in range(len(x)):
    defineLikelyClass(x[i], y[i], True)

In [20]:
sentence = "The coach will be using a new approach"
#"The dog is in the house"
#"The coach will be using a new approach" #<-- HARD

#This will build all of the possible base cases for perspective
#based on research forensics for each given word
def constructPerspective(sentence):
    perspective = []
    tokens = [[_, getLikelyClass(_.lower(), True)] for _ in sentence.split(" ")]
    for token in tokens:
        perspective.append([token[1], [token[0]]])
    return perspective
x = constructPerspective(sentence)
print(x)

[['d', ['The']], ['n', ['coach']], ['t', ['will']], ['v', ['be']], ['v', ['using']], ['d', ['a']], ['adj', ['new']], ['n', ['approach']]]


In [21]:
#determines if the given rule is at the top of a given tree
#Strict disallows the tree from having different neighbors at the top
def inTreeTop(tree, rule, strict = False):
    retVal = False
    strictVal = True
    for topNode in tree:
        if topNode[0] == rule:
            retVal = True
        else:
            strictVal = False
    if strict:
        return (retVal, strictVal)
    return retVal
def pruneTrees(trees):
    pruned = []
    for tree in trees:
        finishedVal = inTreeTop(tree, "cP", strict = True)
        if finishedVal[0] and finishedVal[1]:
            pruned.append(tree)
    return pruned

In [22]:
#Debugging handling, small but out of the tree builder to keep things clean
def dprint(statement, currentLevel, evalLevel):
    if currentLevel >= evalLevel or currentLevel == -1:
        print(statement)

In [23]:
#I am Michael, I love Raquel, I eat protien, etc...
rules = {
    "adjP" : [["adjbar"]],
    "adjbar" : [["adj"], ["advP", "adjbar"]],
    "advP" : [["advbar"]],
    "advbar" : [["adv"], ["advP", "advbar"]],
    "nbar": [["adjP", "nbar"], ["nbar", "pP"], ["n"], ["n", "pP"], ["pn"]],
    "nP" : [["nbar"], ["nP", "conj", "nP"]],
    "dbar" : [["d", "nP"], ["nP"]],
    "dP" : [["dP", "dbar"], ["dbar"], ["dP", "conj", "dP"]],
    "pbar" : [["p", "dP"]],
    "pP" : [["pbar"]],
    "vP" : [["vbar"], ["vP", "conj", "vP"]],
    "vbar" : [["vbar", "pP"], ["vbar", "advP"], ["advP", "vbar"], ["vbar", "dP"], ["v"], ["v", "pP"], ["v", "dP"], ["v", "adjP"], ["v", "vP"]],
    "negbar" : [["neg", "vP"]],
    "negP" : [["negbar"]],
    "t" : [["v"]],
    "tbar" : [["vP"], ["t", "vP"], ["t", "negP"]],
    "tP" : [["dP", "tbar"], ["tP", "conj", "tP"]],
    "cbar" : [["c", "tP"], ["tP"]],
    "cP" : [["cbar"], ["cP", "conj", "cP"]]
}

#This is the big boy itself
#The rule above define what the builder will try
#As of right now, the builder has a few problems
#It refuses to accept verb complements
#It does not account for null items
#As a result it doesn't yet generate movement
#
#Verboseness is for debugging purposes, it works in levels
#0 - Print nothing, smooth, user experience
#1 - Prints things like how many trees are being considered in any one permutation of rules
#2 - ...
#3 - ...
#... - ...
#-1 - PRINTS. EVERYTHING. Be careful what you wish for!!!
def BuildSyntaxTree(x, maxNode = "cP", verbose = 0):
    #Keep track of where matches for each rule occurs, when creating a new node,
    #select the simplest matches first (adjbar before tbar), and the match that occurs later in the tree
    mutating = True
    
    #This is temporary for now -- in the future we will have multiple perspectives, not just one!
    perspectives = [x]
    new_perspectives, final_perspectives = [], []
    exhaustive_perspectives = perspectives.copy()
    
    #Measures tree formation
    deltas = {}
    while mutating:
        new_perspectives = []
        for inp in perspectives:
            matches = {}
            for rule, conditionSet in rules.items():  
                    dprint("Rule exploration", verbose, 5)
                    for conditions in conditionSet:
                        conditionMatch, conditionPosition = 0, 0
                        dprint(rule + " : " + str(conditions), verbose, 5)
                        dprint("Rule matches:", verbose, 6)
                        for i in range(len(inp)):
                            node = inp[i]
                            if node[0] == conditions[conditionMatch]:
                                dprint(rule + ", " + node[0], verbose, 6)
                                if conditionMatch == 0:
                                    conditionPosition = i
                                conditionMatch += 1
                                if conditionMatch == len(conditions):
                                    dprint("Total match for " + rule, verbose, 6)
                                    if rule in matches:
                                        matches[rule] += [(conditionPosition, conditionPosition + conditionMatch)]
                                    else:
                                        matches.update({rule:[(conditionPosition, conditionPosition + conditionMatch)]})
                                    conditionMatch = 0
                            else:
                                conditionMatch = 0
            dprint("Inputs to Matches:\n {} -> {}".format(inp, matches), verbose, 2)
            
            dprint("Rule matches: ", verbose, 3)
            for matchLabel, matchSets in matches.items():
                for matchSet in matchSets:
                    old, new = exhaustive_perspectives.index(inp), -1
                    
                    dprint("{} :- {}".format(matchLabel, matchSet), verbose, 3)
                    subset = inp[matchSet[0]:matchSet[1]]
                    new_perspective = inp[:matchSet[0]] + [[matchLabel, subset]] + inp[matchSet[1]:]
                    
                    if new_perspective not in exhaustive_perspectives:
                        exhaustive_perspectives.append(new_perspective)
                    new = exhaustive_perspectives.index(new_perspective)
                    
                    deltas[(old, new)] = matchLabel
                    
                    if new_perspective not in new_perspectives:
                        new_perspectives.append(new_perspective)
            dprint("Filling in Exaustive and Final arrays...", verbose, 2)
            for perspective in perspectives:
                if perspective not in exhaustive_perspectives:
                    dprint("{} -> exhaustive_perspectives, generically missing".format(perspective), verbose, 3)
                    exhaustive_perspectives.append(perspective)
                if inTreeTop(perspective, maxNode, strict = True) == (True, True) and perspective not in final_perspectives:
                    dprint("{} -> final_perspectives, completed tree".format(perspective), verbose, 3)
                    final_perspectives.append(perspective)
        #Stops searching
        if perspectives == new_perspectives or len(new_perspectives) == 0:
            dprint("Halting, onboarding last trees", verbose, 2)
            for perspective in new_perspectives:
                if perspective not in final_perspectives:
                    dprint("{} -> final_perspectives".format(perspective), verbose, 3)
                    final_perspectives.append(perspectives)
            mutating = False
            dprint("Halted", verbose, 1)
            break
        perspectives = new_perspectives.copy()
        dprint("Permutations in iteration: {}".format(len(perspectives)), verbose, 1)
    #Returns a tuple of:
    #Final list of highest level trees
    #All generated trees
    #Changes, corresponding to exhaustive indicies
    return (final_perspectives, exhaustive_perspectives, deltas)

In [24]:
treeData = BuildSyntaxTree(constructPerspective(sentence), verbose = 1)

Permutations in iteration: 7
Permutations in iteration: 24
Permutations in iteration: 60
Permutations in iteration: 124
Permutations in iteration: 221
Permutations in iteration: 343
Permutations in iteration: 474
Permutations in iteration: 591
Permutations in iteration: 671
Permutations in iteration: 701
Permutations in iteration: 680
Permutations in iteration: 609
Permutations in iteration: 507
Permutations in iteration: 391
Permutations in iteration: 279
Permutations in iteration: 184
Permutations in iteration: 112
Permutations in iteration: 62
Permutations in iteration: 32
Permutations in iteration: 14
Permutations in iteration: 2
Halted


In [25]:
def buildFormationChain(treeData, index):
    if index >= len(treeData[0]):
        return (None, None)
    idx = treeData[1].index(treeData[0][index])
    keychain = []
    formationchain = '{}'.format(idx)
    newKey = True
    while newKey:
        newKey = False
        for key in treeData[2].keys():
            if key[1] == idx:
                idx = key[0]
                keychain.append(key)
                formationchain = '{}->'.format(treeData[2][key]) + formationchain
                newKey = True
                break
    formationchain = '{}->'.format(idx) + formationchain
    return (formationchain, keychain)

In [26]:
buildFormationChain(treeData, 1)

('0->adjbar->adjP->nbar->nbar->nbar->nP->nP->dbar->dbar->dP->dP->vbar->vbar->vP->vbar->vP->tbar->tP->cbar->cP->6078',
 [(6050, 6078),
  (5992, 6050),
  (5885, 5992),
  (5700, 5885),
  (5420, 5700),
  (5023, 5420),
  (4511, 5023),
  (3898, 4511),
  (3217, 3898),
  (2516, 3217),
  (1845, 2516),
  (1254, 1845),
  (780, 1254),
  (437, 780),
  (216, 437),
  (92, 216),
  (32, 92),
  (8, 32),
  (1, 8),
  (0, 1)])

In [27]:
finalTree = pruneTrees(treeData[0])

In [31]:
print("Trees: {}".format(len(finalTree)))
print(treeHeight(finalTree[0]))
finalTree[0]

Trees: 2
13


[['cP',
  [['cbar',
    [['tP',
      [['dP',
        [['dbar', [['d', ['The']], ['nP', [['nbar', [['n', ['coach']]]]]]]]]],
       ['tbar',
        [['t', ['will']],
         ['vP',
          [['vbar',
            [['vbar', [['v', ['be']], ['vP', [['vbar', [['v', ['using']]]]]]]],
             ['dP',
              [['dbar',
                [['d', ['a']],
                 ['nP',
                  [['nbar',
                    [['adjP', [['adjbar', [['adj', ['new']]]]]],
                     ['nbar', [['n', ['approach']]]]]]]]]]]]]]]]]]]]]]]]]

In [34]:
latexKeys = {
    "xbar" : '$\\bar{{{}}}$'
}

def printTree(tree):
    if type(tree) == str:
        return tree + " "
    t = tree[0]
    if type(t) == str:
        return t + " "
    label = t[0]
    if len(label) == 1:
        label = label.upper()
    ploc = label.find('P')
    if ploc > 0:
        if len(label[:ploc]) == 1:
            label = label.upper()
    barloc = label.find('bar')
    if barloc > 0:
        sublabel = label[:barloc]
        if len(sublabel) == 1:
            sublabel = sublabel.upper()
        label = latexKeys["xbar"].format(sublabel)
    if len(t[1]) == 1:
        return '[.{} {}]'.format(label, printTree(t[1]))
    return '[.{} \n\t{} \n\t{}]'.format(label, printTree([t[1][0]]), printTree([t[1][1]]))
    
print(printTree(treeData[0][1]))

[.CP [.$\bar{C}$ [.TP 
	[.DP [.$\bar{D}$ 
	[.D The ] 
	[.NP [.$\bar{N}$ [.N coach ]]]]] 
	[.$\bar{T}$ 
	[.T will ] 
	[.VP [.$\bar{V}$ 
	[.V be ] 
	[.VP [.$\bar{V}$ 
	[.$\bar{V}$ [.V using ]] 
	[.DP [.$\bar{D}$ 
	[.D a ] 
	[.NP [.$\bar{N}$ 
	[.adjP [.$\bar{adj}$ [.adj new ]]] 
	[.$\bar{N}$ [.N approach ]]]]]]]]]]]]]]


In [30]:
#Gives the height of a tree, for those from perspective
def treeHeight(tree):
    if type(tree) == str:
        return 0
    if len(tree) == 1:
        if type(tree[0]) == str:
            return 0
    return 1+max([treeHeight(_[1]) for _ in tree])
    #return max([(0 if len(_) == 1 else max(treeHeight(__) for __ in _[1]) if (type(_) == list) else 0) for _ in tree]) + 1

#i = -10
#print(final_perspectives[0])#(perspectives[i])
#treeHeight(final_perspectives[0])#(perspectives[i])

In [ ]:
for matchLabel, matchSets in matches.items():
    for matchSet in matchSets:
        print("{} :- {}".format(matchLabel, matchSet))
        subset = inp[matchSet[0]:matchSet[1]]
        print(inp[:matchSet[0]] + [[matchLabel, subset]] + inp[matchSet[1]:])

#subset = inp[matches[matchLabel][0][0]:matches[matchLabel][0][1]]
#print(inp[:matches[matchLabel][0][0]] + [[matchLabel, subset]] + inp[matches[matchLabel][0][1]:])


In [33]:
from ete2 import Tree
t = Tree( "((a,b),c);" )
t.render("mytree.png", w=183, units="mm")

ModuleNotFoundError: No module named 'ete2'